In [7]:
from datetime import date
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import csv
import time 

In [8]:
def get_page(browser, link_url):
    values = ['-']*22
    browser.get(link_url)
    name = browser.find_element_by_css_selector('h1.postcard__title')
    try:
        about = browser.find_element_by_css_selector('span.bare-value')
        values[headers.index('About')] = about.text
    except NoSuchElementException:
        about = browser.find_elements_by_css_selector('p.premium-paragraph__text')
        about = about[0].text.replace('\n', ' ')
        values[headers.index('About')] = about
    except NoSuchElementException:
            pass
    values[headers.index('Name')] = name.text
    address_divs = browser.find_elements_by_css_selector('div.profile__address')
    address = ''
    for address_div in address_divs:
        address += address_div.text
    values[headers.index('Address')] = address.replace('Address\n', '').replace('\n', ' ')
    try:
        website = browser.find_element_by_css_selector('a.profile__website__link')
        website_url = website.get_attribute('href')
        values[headers.index('website')] = website_url
    except NoSuchElementException:
            pass
    try:
        admission_bucket = browser.find_element_by_css_selector('#admissions .profile__buckets')
        admission_items = admission_bucket.find_elements_by_css_selector('.scalar, .scalar--three')
        first_label = admission_items[0].find_element_by_css_selector('.scalar__label')
        for admission_item in admission_items:
            label = admission_item.find_element_by_css_selector('.scalar__label')
            try:
                value = admission_item.find_element_by_css_selector('.scalar__value')
                values[headers.index(label.text)] = value.text
            except NoSuchElementException:
                pass
        try:
            url_apply = '-'
            if first_label.text == 'Application Deadline':
                url_apply = browser.find_element_by_css_selector('#admissions .profile__bucket--3 .premium-buttons').find_element_by_link_text('How to Apply').get_attribute('href')
            else:
                url_apply = browser.find_element_by_css_selector('#admissions .profile__bucket--2 .profile__website__link').get_attribute('href')
            values[headers.index('Application Website')] = url_apply
        except NoSuchElementException:
                pass
    except NoSuchElementException:
            pass
    try:
        collapse_profiles_btn = browser.find_element_by_css_selector('.toggle__content__link--profiles')
        browser.execute_script("arguments[0].scrollIntoView(true);", collapse_profiles_btn)
        collapse_profiles_btn.click()
        other_colleges = browser.find_elements_by_css_selector('#admissions .popular-entity-link')
        other_cols = []
        for col in other_colleges:
            other_cols.append(col.text)
        values[headers.index('Students also applied to')] = ', '.join(other_cols)
    except NoSuchElementException:
        pass
    try:
        s_f_ratio_element = browser.find_element_by_css_selector('#academics .profile__bucket--1 .scalar--three')
        s_f_ratio_label = s_f_ratio_element.find_element_by_css_selector('.scalar__label').text
        s_f_ratio = s_f_ratio_element.find_element_by_css_selector('.scalar__value').text
        s_f_ratio = str.replace(s_f_ratio, ':00', '')
        s_f_ratio = str.replace(s_f_ratio, ':01', ':1')
    except NoSuchElementException:
        s_f_ratio = '-'
    finally:
        values[headers.index('Student Faculty Ratio')] = s_f_ratio
    try:
        collapse_majors_btn = browser.find_element_by_css_selector('#majors .toggle__content__link--profiles')
        browser.execute_script("arguments[0].scrollIntoView(true);", collapse_majors_btn)
        collapse_majors_btn.click()
    except NoSuchElementException:
        pass
    majors = []
    major_items = browser.find_elements_by_css_selector('#majors .popular-entity__name')
    for major in major_items:
        majors.append(major.text)
    values[headers.index('Majors')] = ', '.join(majors)
    try:
        students_bucket = browser.find_element_by_css_selector('#students .profile__bucket--1 .blank__bucket')
        students_items = students_bucket.find_elements_by_css_selector('.scalar, .scalar--three')
        for students_item in students_items:
            label = students_item.find_element_by_css_selector('.scalar__label')
            try:
                value = students_item.find_element_by_css_selector('.scalar__value')
                values[headers.index(label.text)] = value.text
            except NoSuchElementException:
                pass
    except NoSuchElementException:
        pass
    try:
        cost_link = browser.find_element_by_css_selector('#cost .expansion-link__text')
        browser.execute_script("arguments[0].scrollIntoView(true);", cost_link)
        cost_link.click()
        try:
            ofs_tuition = browser.find_element_by_css_selector('#sticker-price .profile__bucket--2 .scalar__value').text
            values[headers.index('Out-of-State Tuition')] = ofs_tuition
        except NoSuchElementException:
            pass
    except NoSuchElementException:
            pass
    return values

browser = webdriver.Firefox()
browser2 = webdriver.Firefox()
headers = ['Name', 'About', 'Address', 'website', 'Acceptance Rate', 'SAT Range', 'ACT Range', 'Application Fee', 'SAT/ACT', 'High School GPA', 'Early Decision/Early Action', 'Application Website', 'Students also applied to', 'Majors', 'Full-Time Enrollment', 'Part-Time Undergrads', 'Undergrads Over 25', 'Pell Grant', 'Varsity Athletes', 'Out-of-State Tuition', 'Student Faculty Ratio', 'Application Deadline', 'Accepts Common App']
with open('niche.csv', 'a+', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
#     csv_writer.writerow(headers)
    x = range(1,4)
    for n in x:
        print(n)
        main_url = 'https://www.niche.com/colleges/search/all-colleges/?type=communityCollege&type=tradeSchool'
        if n > 1:
            main_url = main_url + '&page=' + str(n)
        browser.get(main_url)
        names = browser.find_elements_by_css_selector('h2.search-result__title')
        links = browser.find_elements_by_css_selector('a.search-result__link')
#         for name in names:
#             print(name.text)
        for i in range(len(links)):
            if i > 24:
                break
            link_url = links[i].get_attribute('href')
            values = get_page(browser2, link_url)
            csv_writer.writerow(values)
browser.quit()
browser2.quit()
# print(headers, values)
# print(headers)


1
2
3
